In [3]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [9]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************FDqp. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 인적정보 가져오기

In [5]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


### 프롬프트

In [6]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 사용자의 답변 , 사용자가 입력한 주관식 답변 그리고 노인들의 외로움과 한국형 외로움에 대한 연구를 바탕으로 어떤 한국형 외로움을 가지고있는지 설명해주세요. \\
    그 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. \\
    사용자의 이름은 {user.name}, 나이는 {user.age}, 성별은 {user.gender}, 학력은 {user.education}, 결혼 여부는 {user.merry}, 자녀 수는 {user.children}, 종교는 {user.religion}, 소득은 {user.income}, 경제 상태는 {user.economy_states}, 건강 상태는 {user.health_states}입니다. \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요
"""

모델 선언

In [8]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''Answer the question based only on the following context:
{context}

Question: {question}
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
Q: 어떤 상황에서 외로움을 느끼시나요? \\
A: 가장 외로웠던 상황은 가족들과 떨어져 살았을 때입니다. 내가 젊을 때 기러기 아빠였어요. 밤에 혼자 집에 가는데 집에 가도 아무도 없겠구나 싶어서 힘들더라고요. 지금도 가끔 생각나요. \\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke("input_message"))


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************FDqp. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
class chaingpt:
    def __init__(self,api_key, retriever, sys_prompt):
        self.template = sys_prompt + '''Answer the question based only on the following context:
        {context}

        Question: {question}
        '''
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.model = ChatOpenAI(api_key=api_key,model='gpt-4o', temperature=1)
        self.chainmodel = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
        | self.prompt  # 프롬프트 템플릿을 적용합니다.
        | self.model  # 모델을 호출합니다.
        | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
        )
    def invoke(self,input_message):
        return self.chainmodel.invoke(input_message)
    
#ex
api_key = OPENAI_API_KEY
retriever = vectordb.as_retriever()
sys_prompt = """사용자의 외로움을 판단하고, 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. """
gpt = chaingpt(api_key,retriever,sys_prompt)
input_message =  """사용자의 외로움은 뭔가요? 적절한 대화상대가 되어주세요."""
print(gpt.invoke(input_message))
        